# BI output results and speed comparaison
## 1. Continuous variable (model 4.3)
### 1.1. Speed comparaison

In [13]:
import os
import sys
__file__
newPath = os.path.dirname(os.path.abspath(""))
if newPath not in sys.path:
    sys.path.append(newPath)

In [18]:
import time as tm
from src.main import*
## Model m4.3
d = pd.read_csv('../data/Howell1.csv', sep=';')
d = d[d.age > 18]
#self.df["weight.per.g"].pipe(lambda x: (x - x.mean()) / x.std())
d.weight = d.weight - d.weight.mean()
d.age = d.age - d.age.mean()
formula = dict(main1 = 'height ~ Normal(mu,sigma)',
            likelihood = 'mu ~ alpha + beta * weight',
            prior1 = 'sigma ~ Uniform(0,50)',
            prior2 = 'alpha ~ Normal(178,20)',
            prior3 = 'beta ~ Normal(0,1)')    

self = model(formula, df = d, float = 32)
start = tm.time()
self.fit(observed_data = dict(height =d.height.astype('float32').values),
                                           num_results = 2000, num_burnin_steps=500, num_adaptation_steps=400, num_chains=4)   
end = tm.time()    
self.summary(round_to = 'none')     
print(f"BI took: {end - start:.4f} seconds")

2024-03-12 13:16:13.553739: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 13:16:13.572328: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 13:16:13.572536: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 13:16:13.706443: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 13:16:13.706477: I external/local_xla/xla/stream_executor

BI took: 4.6471 seconds


In [41]:
import stan
import nest_asyncio
nest_asyncio.apply()
stan_code = """
data{
  vector[346] height;
  vector[346] weight;
}
parameters{
  real a;
  real<lower=0> b;
  real<lower=0,upper=50> sigma;
}
model{
  vector[346] mu;
  sigma ~ uniform( 0 , 50 );
  b ~ lognormal( 0 , 1 );
  a ~ normal( 178 , 20 );
  for ( i in 1:346 ) {
    mu[i] = a + b* weight[i] ;
  }
  height ~ normal( mu , sigma );
  
}
"""
data = {
  'height': d.height.values,
  'weight': d.weight.values,
}
start = tm.time()
stan_model = stan.build(stan_code, data = data)
fit = stan_model.sample(num_chains=4, num_samples=2000, num_warmup = 500)
end = tm.time()    
df = fit.to_frame()
print(f"Pystan took: {end - start:.4f} seconds")

Building...



Building: 11.1s, done.Messages from stanc:
Warning in '/tmp/httpstan_bzl01w9e/model_qxjp4bwu.stan', line 15, column 20: Argument
    20 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).
Warning in '/tmp/httpstan_bzl01w9e/model_qxjp4bwu.stan', line 15, column 14: Argument
    178 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).
Warning in '/tmp/httpstan_bzl01w9e/model_qxjp4bwu.stan', line 13, column 23: Argument
    50 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).
    in its declaration. These hard constraints are not recommended, for two
    reasons: (a) Except when there are logical or physical constraints, it is
    very unusual for you to be sure that a parameter will fall inside a
    specified range, and (b) The infinite gradient induced by a hard


Pystan took: 11.7146 seconds


### 1.2. Output comparaison 

In [42]:
pd.DataFrame(
    {
        "tfp": self.summary(round_to = 'none')['mean'],
        "pystan": [df.sigma.mean(),  df.b.mean(), df.a.mean()]
    })

,tfp,pystan
sigma[0],5.138569,5.142233
beta[0],0.905371,0.904922
alpha[0],154.649124,154.653531


## 2. Categorical variable (model 5.9)
### 2.1. Speed comparaisons

In [43]:
from src.main import*
self = model()
self.import_csv('../data/milk.csv', sep = ';')
self.df["K"] = self.df["kcal.per.g"].pipe(lambda x: (x - x.mean()) / x.std())
self.index(cols = "clade")
formula = dict(main = 'K ~ Normal(mu,sigma)',
            likelihood = 'mu ~ alpha[index_clade]',
            prior1 = 'alpha~ Normal(0,0.5)',
            prior2 = 'sigma ~ Exponential(1)') 

self.f = formula

start = tm.time()
self.build_model()
self.fit(observed_data = dict(K =self.df.K.astype('float32').values),
                                           num_results = 2000, num_burnin_steps=500, num_adaptation_steps=400, num_chains=4)
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")

2024-03-12 08:41:05.610587: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:41:05.610635: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:41:05.610650: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:41:05.610885: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:41:05.610897: I tensorflow/core/common_runtime/gpu/gpu

BI took: 4.2587 seconds


In [44]:
import stan
import nest_asyncio
nest_asyncio.apply()
stan_code = """
data{
    vector[29] K;
    array[29] int clade_id;
}
parameters{
    vector[4] a;
    real<lower=0> sigma;
}
model{
    vector[29] mu;
    sigma ~ exponential( 1 );
    a ~ normal( 0 , 0.5 );
    for ( i in 1:29 ) {
        mu[i] = a[clade_id[i]];
    }
    K ~ normal( mu , sigma );
    
}
"""
data = {
  'clade_id': self.df.index_clade.values+1,
  'K': self.df.K.values,
}
start = tm.time()
stan_model = stan.build(stan_code, data = data)
fit = stan_model.sample(num_chains=4, num_samples=2000, num_warmup = 500)
end = tm.time()    
df = fit.to_frame()
print(f"Pystan took: {end - start:.4f} seconds")

Building...



Building: 11.5s, done.Sampling:   0%
Sampling:  25% (2500/10000)
Sampling:  50% (5000/10000)
Sampling:  75% (7500/10000)
Sampling: 100% (10000/10000)
Sampling: 100% (10000/10000), done.
Messages received during sampling:
  Gradient evaluation took 2e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.2 seconds.
  Adjust your expectations accordingly!
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: normal_lpdf: Scale parameter is 0, but must be positive! (in '/tmp/httpstan_wva56780/model_utzoleag.stan', line 17, column 4 to column 29)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Gradient evaluation took 2.1e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would

Pystan took: 11.8828 seconds


### 2.2. Output comparaison

In [45]:
pd.DataFrame(
    {
        "tfp": self.summary(round_to = 'none')['mean'],
        "pystan": [df.sigma.mean(),  df['a.1'].mean(), df['a.2'].mean(), df['a.3'].mean(), df['a.4'].mean()]
    })

,tfp,pystan
sigma[0],0.797389,0.800678
alpha[0],-0.463416,-0.463840
alpha[1],0.348681,0.345072
alpha[2],0.633578,0.637753
alpha[3],-0.550391,-0.546003


## 3. Continuous interactions terms (model 8.3)
### 3.1. Speed comparaisons

In [46]:
from src.main import*
d = pd.read_csv('../data/tulips.csv', sep = ';')
d["blooms_std"] = d.blooms / d.blooms.max()
d["water_cent"] = d.water - d.water.mean()
d["shade_cent"] = d.shade - d.shade.mean()

formula = dict(
            main = 'blooms_std ~ Normal( mu , sigma ) ',
            likelihood ='mu ~ a + bw*water_cent + bs*shade_cent + bws*water_cent*shade_cent' ,
            prior1 = 'a ~ Normal( 0.5 , 0.25 ) ',
            prior2 = 'bw ~ Normal( 0 , 0.25 ) ',
            prior3 = 'bs ~ Normal( 0 , 0.25 ) ',
            prior4 = 'bws ~ Normal( 0 , 0.25 ) ',
            prior5 = 'sigma ~ Exponential( 1 )',
            )
start = tm.time()
m8_5 = model(formula, d)
m8_5.fit(observed_data = dict(blooms_std =d.blooms_std.astype('float32').values),
                                           num_results = 2000, num_burnin_steps=500, num_adaptation_steps=400, num_chains=4)
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")

2024-03-12 08:41:28.892398: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:41:28.892472: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:41:28.892486: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:41:28.892659: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:41:28.892672: I tensorflow/core/common_runtime/gpu/gpu

BI took: 7.1625 seconds


In [47]:
import stan
import nest_asyncio
nest_asyncio.apply()

stan_code = """
data{
    vector[27] blooms_std;
    array[27] int shade_cent;
    array[27] int water_cent;
}
parameters{
    real a;
    real bw;
    real bs;
    real bws;
    real<lower=0> sigma;
}
model{
    vector[27] mu;
    sigma ~ exponential( 1 );
    bws ~ normal( 0 , 0.25 );
    bs ~ normal( 0 , 0.25 );
    bw ~ normal( 0 , 0.25 );
    a ~ normal( 0.5 , 0.25 );
    for ( i in 1:27 ) {
        mu[i] = a + bw * water_cent[i] + bs * shade_cent[i] + bws * water_cent[i] * shade_cent[i];
    }
    
    blooms_std ~ normal( mu , sigma );
}
"""

data = {
    'blooms_std' : d["blooms_std"].values,
    "water_cent": d["water_cent"].values.astype(int),
    "shade_cent": d["shade_cent"].values.astype(int),
}
start = tm.time()
stan_model = stan.build(stan_code, data = data)
fit = stan_model.sample(num_chains=4, num_samples=2000, num_warmup = 500)
end = tm.time()    
df = fit.to_frame()
print(f"Pystan took: {end - start:.4f} seconds")

Building...



Building: 11.3s, done.Sampling:   0%
Sampling:  25% (2500/10000)
Sampling:  50% (5000/10000)
Sampling:  75% (7500/10000)
Sampling: 100% (10000/10000)
Sampling: 100% (10000/10000), done.
Messages received during sampling:
  Gradient evaluation took 1.7e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.17 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 1.6e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.16 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 1.7e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.17 seconds.
  Adjust your expectations accordingly!
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: normal_lpdf: Scale parameter is 0, but must be positive! (in '/tmp/httpstan_5kkvti7s/model_7gq22chx.stan', line 25, column 4 to column 38)
  If this warni

Pystan took: 11.7797 seconds


### 3.2. Output comparaison (PB estimation)

In [48]:
pd.DataFrame(
    {
        "tfp": m8_5.summary(round_to='none')['mean'],
        "pystan": [df.sigma.mean(),  df['bws'].mean(), df['bw'].mean(), df['bs'].mean(), df['a'].mean()]
    })

,tfp,pystan
sigma[0],0.095120,0.142249
bws[0],-0.223282,-0.142495
bw[0],0.010972,0.206738
bs[0],0.078516,-0.112914
a[0],0.393192,0.358084


## 4. Binomial model
### 4.1. Output comparaisons

In [49]:
from src.main import*
d = pd.read_csv('../data/chimpanzees.csv', sep = ';')
d["treatment"] = 1 + d.prosoc_left + 2 * d.condition
d["side"] = d.prosoc_left  # right 0, left 1
d["cond"] = d.condition  # no partner 0, partner 1

d_aggregated = (
    d.groupby(["treatment", "actor", "side", "cond"])["pulled_left"].sum().reset_index()
)
d_aggregated.rename(columns={"pulled_left": "left_pulls"}, inplace=True)
d_aggregated["actor_id"] = d_aggregated["actor"].values - 1

formula = dict(
    main = 'pulled_left ~ Binomial( 1 , logits = p )' ,
    likelihood = 'p ~ a' ,
    prior1 = 'a ~ Normal( 0 , 10 )'
)
start = tm.time()
m11_1 = model(formula, d)
m11_1.fit(observed_data = dict(pulled_left =d.pulled_left.astype('float32').values),
                                           num_results = 2000, num_burnin_steps=500, num_adaptation_steps=400, num_chains=4)
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")

2024-03-12 08:41:56.291437: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:41:56.291516: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:41:56.291531: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:41:56.291699: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:41:56.291712: I tensorflow/core/common_runtime/gpu/gpu

BI took: 3.9502 seconds


In [50]:
import stan
import nest_asyncio
nest_asyncio.apply()

stan_code = """
data{
    array[504] int pulled_left;
}
parameters{
    real a;
}
model{
    real p;
    a ~ normal( 0 , 10 );
    p = a;
    p = inv_logit(p);
    pulled_left ~ binomial( 1 , p );
    
}
"""

data = {
    'pulled_left' : d["pulled_left"].values.astype(int)
}
start = tm.time()
stan_model = stan.build(stan_code, data = data)
fit = stan_model.sample(num_chains=4, num_samples=2000, num_warmup = 500)
end = tm.time()    
df = fit.to_frame()
print(f"Pystan took: {end - start:.4f} seconds")

Building...



Building: 9.5s, done.Sampling:   0%
Sampling:  25% (2500/10000)
Sampling:  50% (5000/10000)
Sampling:  75% (7500/10000)
Sampling: 100% (10000/10000)
Sampling: 100% (10000/10000), done.
Messages received during sampling:
  Gradient evaluation took 5e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.05 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 6e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.06 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 5e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.05 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 4e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.04 seconds.
  Adjust your expectations accordingly!


Pystan took: 9.7943 seconds


### 4.2. Output comparaison

In [51]:
pd.DataFrame(
    {
        "tfp": m11_1.summary(round_to='none')['mean'],
        "pystan": [df.a.mean()]
    })

,tfp,pystan
a[0],0.321594,0.323123


##  5. Binomial with indices
### 5.1.Speed comparaisons

In [52]:
from src.main import*
d = pd.read_csv('../data/chimpanzees.csv', sep = ';')
d.actor = d.actor - 1
d["treatment"] = d.prosoc_left + 2 * d.condition
d[["actor", "prosoc_left", "condition", "treatment"]]

formula = dict(
    main = 'pulled_left ~ Binomial(1 , p )' ,
    likelihood = 'p ~ a[actor] + b[treatment]' ,
    prior1 = 'a ~ Normal(0, 1.5)',
    prior2 = 'b ~ Normal(0, 0.5)'
)

start = tm.time()
m11_4 = model(formula, d, float = 32)
m11_4.fit(observed_data = dict(pulled_left =d.pulled_left.astype('float32').values),
                                           num_results = 2000, num_burnin_steps=500, num_adaptation_steps=400, num_chains=4)
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")


2024-03-12 08:42:21.132350: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:42:21.132419: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:42:21.132433: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:42:21.132622: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:42:21.132634: I tensorflow/core/common_runtime/gpu/gpu

BI took: 6.8511 seconds


In [53]:
import stan
import nest_asyncio
nest_asyncio.apply()

stan_code = """
data{
    array[504] int pulled_left;
    array[504] int treatment;
    array[504] int actor;
}
parameters{
    vector[7] a;
    vector[4] b;
}
model{
    vector[504] p;
    b ~ normal( 0 , 0.5 );
    a ~ normal( 0 , 1.5 );
    for ( i in 1:504 ) {
        p[i] = a[actor[i]] + b[treatment[i]];
        p[i] = inv_logit(p[i]);
    }
    pulled_left ~ binomial( 1 , p );
}
generated quantities{
    vector[504] log_lik;
    vector[504] p;
    for ( i in 1:504 ) {
        p[i] = a[actor[i]] + b[treatment[i]];
        p[i] = inv_logit(p[i]);
    }
    for ( i in 1:504 ) log_lik[i] = binomial_lpmf( pulled_left[i] | 1 , p[i] );
    
}
"""

data = {
    'pulled_left' : d["pulled_left"].values.astype(int),
    'treatment' : d["treatment"].values.astype(int) +1,
    'actor' : d["actor"].values.astype(int) +1
}

start = tm.time()
stan_model = stan.build(stan_code, data = data)
fit = stan_model.sample(num_chains=4, num_samples=2000, num_warmup = 500)
end = tm.time()    
df = fit.to_frame()
print(f"Pystan took: {end - start:.4f} seconds")

Building...



Building: 11.6s, done.Sampling:   0%
Sampling:  25% (2500/10000)
Sampling:  50% (5000/10000)
Sampling:  75% (7500/10000)
Sampling: 100% (10000/10000)
Sampling: 100% (10000/10000), done.
Messages received during sampling:
  Gradient evaluation took 3.9e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.39 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 4.4e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.44 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 4.4e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.44 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 4.3e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.43 seconds.
  Adjust your expectations accordingly!


Pystan took: 15.2016 seconds


### 5.2. Output comparaison

In [54]:
pd.DataFrame(
    {
        "tfp": m11_4.summary(round_to='none')['mean'],
        "pystan": [df['b.1'].mean(), df['b.2'].mean(), df['b.3'].mean(), df['b.4'].mean(),
                   df['a.1'].mean(), df['a.2'].mean(), df['a.3'].mean(), df['a.4'].mean(), df['a.5'].mean(), df['a.6'].mean(), df['a.7'].mean()]
    })

,tfp,pystan
b[0],-0.034848,-0.045185
b[1],0.475208,0.475823
b[2],-0.386461,-0.387507
b[3],0.366866,0.364700
a[0],-0.444979,-0.443071
a[1],3.917305,3.890319
a[2],-0.739946,-0.744365
a[3],-0.741094,-0.743383
a[4],-0.447257,-0.443050
a[5],0.482430,0.483948


## 6. Poisson
### 6.1. Speed comparaison

In [55]:
from src.main import*
d = pd.read_csv('../data/Kline.csv', sep = ';')
d["P"] = d.population.pipe(np.log).pipe(lambda x: (x - x.mean()) / x.std())
d["cid"] = (d.contact == "high").astype(int)
d['pLog'] = tf.math.log(d.P).numpy()
formula = dict(main = 'total_tools ~ Poisson(log_rate = lambda)',
               likelihood = 'lambda ~ alpha[cid] + beta[cid]*P',
               prior1 = 'alpha ~ Normal(3,0.5)',
               prior2 = 'beta ~ Normal(0,0.2)')
start = tm.time()
m11_10 = model(formula, d)
m11_10.fit(observed_data = dict(total_tools =d.total_tools.astype('float32').values),
                                           num_results = 2000, num_burnin_steps=500, num_adaptation_steps=400, num_chains=4)
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")

2024-03-12 08:43:18.468534: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:43:18.468588: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:43:18.468601: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:43:18.468813: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:43:18.468826: I tensorflow/core/common_runtime/gpu/gpu

BI took: 4.6212 seconds


In [56]:
import stan
import nest_asyncio
nest_asyncio.apply()

stan_code = """ 
data{
    array[10] int T;
    vector[10] P;
    array[10] int cid;
}
parameters{
    vector[2] a;
    vector[2] b;
}
model{
    vector[10] lambda;
    b ~ normal( 0 , 0.2 );
    a ~ normal( 3 , 0.5 );
    for ( i in 1:10 ) {
       lambda[i] = a[cid[i]] + b[cid[i]] * P[i];
       lambda[i] = exp(lambda[i]);
    }
    T ~ poisson( lambda );
}
generated quantities{
    vector[10] log_lik;
    vector[10] lambda;
    for ( i in 1:10 ) {
        lambda[i] = a[cid[i]] + b[cid[i]] * P[i];
        lambda[i] = exp(lambda[i]);
    }
    for ( i in 1:10 ) log_lik[i] = poisson_lpmf( T[i] | lambda[i]);
    
}
"""

data = {
    'T' : d["total_tools"].values.astype(int),
    'P' : d["P"].values.astype(float),
    'cid' : d["cid"].values.astype(int) +1
}

start = tm.time()
stan_model = stan.build(stan_code, data = data)
fit = stan_model.sample(num_chains=4, num_samples=2000, num_warmup = 500)
end = tm.time()    
df = fit.to_frame()
print(f"Pystan took: {end - start:.4f} seconds")

Building...



Building: 11.8s, done.Sampling:   0%
Sampling:  25% (2500/10000)
Sampling:  50% (5000/10000)
Sampling:  75% (7500/10000)
Sampling: 100% (10000/10000)
Sampling: 100% (10000/10000), done.
Messages received during sampling:
  Gradient evaluation took 6e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.06 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 7e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.07 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 6e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.06 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 6e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.06 seconds.
  Adjust your expectations accordingly!


Pystan took: 12.1948 seconds


### 6.2. Output comparaison 

In [57]:
pd.DataFrame(
    {
        "tfp": m11_10.summary(round_to='none')['mean'],
        "pystan": [df['b.1'].mean(), df['b.2'].mean(), 
                   df['a.1'].mean(), df['a.2'].mean()]
    })

,tfp,pystan
beta[0],0.377117,0.377000
beta[1],0.191245,0.193386
alpha[0],3.319345,3.319936
alpha[1],3.605054,3.609570


## 7. Negative binomial
### 7.1. Speed comparaison

In [79]:
from src.main import*
num_days = 30
y = tfd.Poisson(rate=1.5).sample((num_days,))
num_weeks = 4
y_new = tfd.Poisson(rate=0.5 * 7).sample((num_weeks,))
y_all = np.concatenate([y, y_new])
exposure = np.concatenate([np.repeat(1, 30), np.repeat(7, 4)])
monastery = np.concatenate([np.repeat(0, 30), np.repeat(1, 4)])
d = pd.DataFrame.from_dict(dict(y=y_all, days=exposure, monastery=monastery))
d["log_days"] = d.days.pipe(np.log)


formula = dict(main = 'y ~ Poisson(log_rate = lambda)',
               likelihood = 'lambda ~ log_days + alpha +  beta * monastery',
               prior1 = 'alpha ~ Normal(0,1)',
               prior2 = 'beta ~ Normal(0,1)')
start = tm.time()
m11_12 = model(formula, d, float=32)
m11_12.fit(observed_data = dict(y =d.y.astype('float32').values),
                                           num_results = 2000, num_burnin_steps=500, num_adaptation_steps=400, num_chains=4)
end = tm.time()
print(f"BI took: {end - start:.4f} seconds")

2024-03-12 09:06:24.196772: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 09:06:24.196850: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 09:06:24.196869: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 09:06:24.197046: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 09:06:24.197058: I tensorflow/core/common_runtime/gpu/gpu

BI took: 4.3948 seconds


In [81]:
import stan
import nest_asyncio
nest_asyncio.apply()

stan_code =""" 
data{
    array[34] int days;
    array[34] int y;
    array[34] int monastery;
    vector[34] log_days;
}
parameters{
    real a;
    real b;
}
model{
    vector[34] lambda;
    b ~ normal( 0 , 1 );
    a ~ normal( 0 , 1 );
    for ( i in 1:34 ) {
        lambda[i] = log_days[i] + a + b * monastery[i];
        lambda[i] = exp(lambda[i]);
    }
    
    y ~ poisson( lambda );
    
}
"""

data = {
    'days' : d["days"].values.astype(int),
    'y' : d["y"].values.astype(int),
    'monastery' : d["monastery"].values.astype(int) +1,
    'log_days' : d["log_days"].values.astype(float),
}

start = tm.time()
stan_model = stan.build(stan_code, data = data)
fit = stan_model.sample(num_chains=4, num_samples=2000, num_warmup = 500)
end = tm.time()    
df = fit.to_frame()
print(f"Pystan took: {end - start:.4f} seconds")

Building...



Building: 11.0s, done.Sampling:   0%
Sampling:  25% (2500/10000)
Sampling:  50% (5000/10000)
Sampling:  75% (7500/10000)
Sampling: 100% (10000/10000)
Sampling: 100% (10000/10000), done.
Messages received during sampling:
  Gradient evaluation took 7e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.07 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 7e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.07 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 1.4e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.14 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 6e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.06 seconds.
  Adjust your expectations accordingly!


Pystan took: 11.3787 seconds


### Output comparaison (PB estimation)

In [84]:
pd.DataFrame(
    {
        "tfp": m11_12.summary(round_to='none')['mean'],
        "pystan": [df['b'].mean(), df['a'].mean()]
    })

,tfp,pystan
beta[0],-1.015814,-0.904373
alpha[0],0.294309,1.150375


## 8. Multinomial

## 9. Beta binomial
### 9.1. Speed comparaison

In [73]:
from src.main import*
d = pd.read_csv('../data/UCBadmit.csv', sep = ';')
d["gid"] = (d["applicant.gender"] != "male").astype(int)
len(d.applications)
formula = dict(main = 'y ~ BetaBinomial(12, concentration1 = pbar, concentration0 = theta)',
               likelihood = 'pbar ~ sigmoid(a[gid])',
               likelihood2 = 'theta ~ phi + -2.0',
               prior1 = 'a ~ Normal(0.,1.5)',
               prior2 = 'phi ~ Exponential(1)'
               )
start = tm.time()
m12_1 = model(formula, d)
m12_1.fit(observed_data = dict(y =d.admit.astype('float32').values),
                                           num_results = 1000, num_burnin_steps=500, num_adaptation_steps=400, num_chains=4)
end = tm.time()
print(f"BI took: {end - start:.4f} seconds")

2024-03-12 08:59:11.428620: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:59:11.428674: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:59:11.428687: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:59:11.428890: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:59:11.428902: I tensorflow/core/common_runtime/gpu/gpu

BI took: 3.5001 seconds


In [74]:
import stan
import nest_asyncio
nest_asyncio.apply()
stan_code =""" 
data{
    array[12] int N;
    array[12] int A;
    array[12] int gid;
}
parameters{
    vector[2] a;
    real<lower=0> phi;
}
transformed parameters{
    real theta;
    theta = phi + 2;
}
model{
    vector[12] pbar;
    phi ~ exponential( 1 );
    a ~ normal( 0 , 1.5 );
    for ( i in 1:12 ) {
        pbar[i] = a[gid[i]];
        pbar[i] = inv_logit(pbar[i]);
    }
    A ~ beta_binomial( N , pbar*theta , (1-pbar)*theta );    
}
"""

data = {
    'A' : d["admit"].values.astype(int),
    'N' : d["applications"].values.astype(int),
    'gid' : d["gid"].values.astype(int) +1,
}

start = tm.time()
stan_model = stan.build(stan_code, data = data)
fit = stan_model.sample(num_chains=4, num_samples=2000, num_warmup = 500)
end = tm.time()    
df = fit.to_frame()
print(f"Pystan took: {end - start:.4f} seconds")

Building...



Building: 12.3s, done.Sampling:   0%
Sampling:  25% (2500/10000)
Sampling:  50% (5000/10000)
Sampling:  75% (7500/10000)
Sampling: 100% (10000/10000)
Sampling: 100% (10000/10000), done.
Messages received during sampling:
  Gradient evaluation took 8e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.08 seconds.
  Adjust your expectations accordingly!
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: beta_binomial_lpmf: Second prior sample size parameter[1] is 0, but must be positive finite! (in '/tmp/httpstan_r_32hiex/model_aokd6ccs.stan', line 23, column 4 to column 57)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Gradient evaluation took 9e-06 seconds
  1000 transitions using 10 

Pystan took: 12.7666 seconds


### 9.2. Output comparaison (PB estimation)

In [78]:
pd.DataFrame(
    {
        "tfp": m12_1.summary(round_to='none')['mean'],
        "pystan": [df['phi'].mean(), df['a.1'].mean(), df['a.2'].mean()]
    })

,tfp,pystan
phi[0],0.515661,1.016125
a[0],0.249002,-0.437358
a[1],-0.622455,-0.326787


## 10. Zero inflated outcomes
### 10.1. Speed comparaison

In [65]:
from src.main import*
import random
random.seed(42)
# Define parameters
prob_drink = 0.2  # 20% of days
rate_work = 1     # average 1 manuscript per day

# sample one year of production
N = 365

np.random.seed(365)
drink = np.random.binomial(1, prob_drink, N)
y = (1 - drink) * np.random.poisson(rate_work, N)
d = pd.DataFrame(y)
formula = dict(main = 'y ~ ZeroInflatedNegativeBinomial(total_count = 365, inflated_loc_logits = p, logits = AL)',
               likelihood = "p ~ ap",
               likelihood2 = "AL ~ tf.math.log(al)",
               prior1 = 'ap ~ Normal(-1.5 , 1)',
               prior2 = 'al ~ Normal(1,0.5)'
               )

start = tm.time()
m12_3 = model()
m12_3 = model(formula)       
m12_3.fit(observed_data = dict(y = d.iloc[:,0].astype('float32').values),
                                           num_results = 2000, num_burnin_steps=500, num_adaptation_steps=400, num_chains=4)
end = tm.time()
print(f"BI took: {end - start:.4f} seconds")

2024-03-12 08:56:24.978128: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:56:24.978175: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:56:24.978188: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:56:24.978344: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:56:24.978355: I tensorflow/core/common_runtime/gpu/gpu

BI took: 4.8690 seconds


In [69]:
import stan
import nest_asyncio
nest_asyncio.apply()
stan_code = """ 
data{
    array[365] int y;
}
parameters{
    real ap;
    real al;
}
model{
    real p;
    real lambda;
    al ~ normal( 1 , 0.5 );
    ap ~ normal( -1.5 , 1 );
    lambda = al;
    lambda = exp(lambda);
    p = ap;
    p = inv_logit(p);
    for ( i in 1:365 ) {
        if ( y[i]==0 )
            target += log_mix( p , 0 , poisson_lpmf(0|lambda) );
        if ( y[i] > 0 )
            target += log1m( p ) + poisson_lpmf(y[i] | lambda );
    }
}
"""
data = {
    'y' :d.iloc[:,0].values.astype(int)
}
start = tm.time()
stan_model = stan.build(stan_code, data = data)
fit = stan_model.sample(num_chains=4, num_samples=2000, num_warmup = 500)
end = tm.time()    
df = fit.to_frame()
print(f"Pystan took: {end - start:.4f} seconds")

Building...



Building: 9.8s, done.Sampling:   0%
Sampling:  25% (2500/10000)
Sampling:  50% (5000/10000)
Sampling:  75% (7500/10000)
Sampling: 100% (10000/10000)
Sampling: 100% (10000/10000), done.
Messages received during sampling:
  Gradient evaluation took 5.2e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.52 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 5.2e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.52 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 7.1e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.71 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 5.5e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.55 seconds.
  Adjust your expectations accordingly!


Pystan took: 11.2434 seconds


### 10.2. Output comparaison (PB estimation)

In [72]:
pd.DataFrame(
    {
        "tfp": m12_3.summary(round_to='none')['mean'],
        "pystan": [df['ap'].mean(), df['al'].mean()]
    })

,tfp,pystan
ap[0],-0.687577,-1.352273
al[0],0.381126,0.111060


## 11. Varying interceps
### 11.1. Speed comparaison

In [73]:
from src.main import*
import time as tm
d = pd.read_csv('../data/reedfrogs.csv', sep = ';')
d["tank"] = np.arange(d.shape[0])
formula = dict(main = 'y ~ Binomial(total_count = density, logits = p)',
               likelihood = 'p ~ alpha[tank]', 
               prior = 'alpha ~ Normal(a_bar, sigma)',
               prior1 = 'a_bar ~ Normal(0.,1.5)',
               prior2 = 'sigma ~ Exponential(1)'
               )

start = tm.time()   
m13_2 = model(formula, d, float=32)
num_chains = 4
samples = m13_2.sample(num_chains)

# Reshape samples to desired shapes
samples['a_bar'] = tf.ones(shape=(4,))
samples['sigma'] = tf.ones(shape=(4,))
samples['alpha'] = tf.ones(shape=(4, 48))

observed_data = dict(y =d.surv.astype('float32').values)
for k in observed_data.keys():
    samples.pop(k)
init_state = list(samples.values())
init_state

m13_2.fit(observed_data = observed_data, num_chains= num_chains, init = init_state, parallel_iterations = 4, 
          num_leapfrog_steps = 3, num_results = 500, 
          step_size=0.1)
end = tm.time()    
print(f"Pystan took: {end - start:.4f} seconds")
#m13_2 = model(formula, d, float=32)
#m = {}
#m['a_bar']= tfd.Normal(0.,1.5, name = 'prior1')
#m['sigma']= tfd.Exponential(1, name = 'prior2')
#m['alpha']= lambda a_bar, sigma: tfd.Sample(tfd.Normal(a_bar,sigma), sample_shape = 48)
#m['y']= lambda alpha : tfd.Independent(tfd.Binomial(total_count= d.density.astype('float32').values,logits= tf.squeeze(tf.gather(alpha,tf.cast(d.tank.astype('float32').values, dtype=tf.int32), axis = -1)), name ='main'), reinterpreted_batch_ndims=1)
#
#m13_2.tensor = tfd.JointDistributionNamed(m)
#m13_2.fit(observed_data = observed_data, num_chains= 4, parallel_iterations = 4, 
#          num_leapfrog_steps = 3, num_results = 500, 
#          step_size=0.1)
#
#m13_2 = model(formula, d, float=32)
#m = {}
#m['a_bar']= tfd.Normal(0.,1.5, name = 'prior1')
#m['sigma']= tfd.Exponential(1, name = 'prior2')
#m['alpha']= lambda a_bar, sigma: tfd.Sample(tfd.Normal(a_bar,sigma), sample_shape = 48)
#m['y']= lambda alpha : tfd.Independent(tfd.Binomial(total_count= d.density.astype('float32').values,logits= alpha, name ='main'), reinterpreted_batch_ndims=1)
#
#m13_2.tensor = tfd.JointDistributionNamed(m)
#m13_2.fit(observed_data = observed_data, num_chains= 4, parallel_iterations = 4, 
#          num_leapfrog_steps = 3, num_results = 500, 
#          step_size=0.1)
#

2024-03-12 14:21:58.003696: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 14:21:58.003744: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 14:21:58.003757: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 14:21:58.004107: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 14:21:58.004120: I tensorflow/core/common_runtime/gpu/gpu

Pystan took: 4.4995 seconds


In [74]:
import stan
import nest_asyncio
nest_asyncio.apply()
stan_code = """ 
data{
    array[48] int N;
    array[48] int S;
    array[48] int tank;
}
parameters{
    vector[48] a;
    real a_bar;
    real<lower=0> sigma;
}
model{
    vector[48] p;
    sigma ~ exponential( 1 );
    a_bar ~ normal( 0 , 1.5 );
    a ~ normal( a_bar , sigma );
    for ( i in 1:48 ) {
        p[i] = a[tank[i]];
        p[i] = inv_logit(p[i]);
    }
    S ~ binomial( N , p );
}
generated quantities{
    vector[48] log_lik;
    vector[48] p;
    for ( i in 1:48 ) {
        p[i] = a[tank[i]];
        p[i] = inv_logit(p[i]);
    }
    for ( i in 1:48 ) log_lik[i] = binomial_lpmf( S[i] | N[i] , p[i] );
    
}
"""
data = {
    'S' : d['surv'].values.astype(int),
    'N' : d['density'].values.astype(int),
    'tank' : d['tank'].values.astype(int)+1,
}
start = tm.time()
stan_model = stan.build(stan_code, data = data)
fit = stan_model.sample(num_chains=4, num_samples=2000, num_warmup = 500)
end = tm.time()    
df = fit.to_frame()
print(f"Pystan took: {end - start:.4f} seconds")

Building...



Building: 12.7s, done.Sampling:   0%
Sampling:  25% (2500/10000)
Sampling:  50% (5000/10000)
Sampling:  75% (7500/10000)
Sampling: 100% (10000/10000)
Sampling: 100% (10000/10000), done.
Messages received during sampling:
  Gradient evaluation took 1.9e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.19 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 1.4e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.14 seconds.
  Adjust your expectations accordingly!
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: normal_lpdf: Scale parameter is 0, but must be positive! (in '/tmp/httpstan_o89kziwc/model_4vp4dja7.stan', line 16, column 4 to column 32)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your

Pystan took: 14.1654 seconds


### 11.3 Output comparaison (instable output from BI)

In [72]:
pd.DataFrame(
    {
        "tfp": m13_2.summary(round_to='none')['mean'],
        "pystan": [df['sigma'].mean(), df['a_bar'].mean(),
                   df['a.1'].mean(), df['a.2'].mean(),
                   df['a.3'].mean(), df['a.4'].mean(),
                   df['a.5'].mean(), df['a.6'].mean(),
                   df['a.7'].mean(), df['a.8'].mean(),
                   df['a.9'].mean(), df['a.10'].mean(),
                   df['a.11'].mean(), df['a.12'].mean(),
                   df['a.13'].mean(), df['a.14'].mean(),
                   df['a.15'].mean(), df['a.16'].mean(),
                   df['a.17'].mean(), df['a.18'].mean(),
                   df['a.19'].mean(), df['a.20'].mean(),
                   df['a.21'].mean(), df['a.22'].mean(),
                   df['a.23'].mean(), df['a.24'].mean(),
                   df['a.25'].mean(), df['a.26'].mean(),
                   df['a.27'].mean(), df['a.28'].mean(),
                   df['a.29'].mean(), df['a.30'].mean(),
                   df['a.31'].mean(), df['a.32'].mean(),
                   df['a.33'].mean(), df['a.34'].mean(),
                   df['a.35'].mean(), df['a.36'].mean(),
                   df['a.37'].mean(), df['a.38'].mean(),
                   df['a.39'].mean(), df['a.40'].mean(),
                   df['a.41'].mean(), df['a.42'].mean(),
                   df['a.43'].mean(), df['a.44'].mean(),
                   df['a.45'].mean(), df['a.46'].mean(),
                   df['a.47'].mean(), df['a.48'].mean()
                   ]
    })

,tfp,pystan
sigma,1.610350,1.618321
a_bar,1.344235,1.349615
alpha[0],1.962730,2.136232
alpha[1],3.078145,3.060428
alpha[2],0.968580,1.011729
alpha[3],2.958655,3.072418
alpha[4],1.982622,2.134639
alpha[5],2.308683,2.157695
alpha[6],3.035657,3.075069
alpha[7],2.666209,2.145086


## 12. Varying effects
### 12.1. Speed comparaison

In [140]:
from src.main import*
from tensorflow_probability import bijectors as tfb
a = 3.5  # average morning wait time
b = -1  # average difference afternoon wait time
sigma_a = 1  # std dev in intercepts
sigma_b = 0.5  # std dev in slopes
rho = -0.7  # correlation between intercepts and slopes
Mu = tf.constant([a, b])
cov_ab = sigma_a * sigma_b * rho
Sigma = tf.constant([[sigma_a ** 2, cov_ab], [cov_ab, sigma_b ** 2]])
tf.transpose(tf.reshape(tf.constant([1, 2, 3, 4]), (2, 2)))
sigmas = tf.constant([sigma_a, sigma_b])  # standard deviations
Rho = tf.constant([[1, rho], [rho, 1]])  # correlation matrix

# now matrix multiply to get covariance matrix
Sigma = tf.linalg.tensor_diag(sigmas) @ Rho @ tf.linalg.tensor_diag(sigmas)
Sigma
N_cafes = 20

def build_vary_effects():
    _seed = 5
    tf.random.set_seed(_seed)

    seed = tfp.util.SeedStream(_seed, salt="vary_effects")

    Mu = tf.constant([a, b])

    vary_effects = tfd.MultivariateNormalTriL(
        loc=Mu, scale_tril=tf.linalg.cholesky(Sigma)
    ).sample((N_cafes,), seed=seed())

    return vary_effects

vary_effects = build_vary_effects()
a_cafe = vary_effects[:, 0]
b_cafe = vary_effects[:, 1]
N_visits = 10
afternoon = np.tile(np.arange(2), N_visits * N_cafes // 2)
cafe_id = np.repeat(np.arange(N_cafes), N_visits)

def generate_data_frame():
    sigma = 0.5  # std dev within cafes

    _seed = 22
    tf.random.set_seed(_seed)

    seed = tfp.util.SeedStream(_seed, salt="generate_data_frame")

    mu = tf.gather(a_cafe, cafe_id) + tf.gather(b_cafe, cafe_id) * afternoon

    wait = tfd.Normal(loc=mu, scale=sigma).sample(seed=seed())
    d = pd.DataFrame(dict(cafe=cafe_id, afternoon=afternoon, wait=wait))

    return d
d = generate_data_frame()

d.head()

formula = dict(main = 'y ~ Normal(mu, sigma)',
likelihood1 = 'mu ~ a_cafe_b_cafe[cafe] + a_cafe_b_cafe[cafe]*afternoon',
prior0 = 'a_cafe_b_cafe ~ MultivariateNormalTriL(concat([alpha, beta],axis=-1), LinearOperatorDiag(sigma_alpha_beta).matmul(Rho))',
prior1 = 'sigma ~ Exponential(1)',
prior2 = 'sigma_alpha_beta ~ Exponential(1)',
prior3 = 'alpha ~ Normal(5,2)',
prior4 = 'beta ~ Normal(-1,0.5)',
prior5 = 'Rho ~ LKJ(2,2)',
)

start = tm.time()
m14_1 = model(formula, d)
init_rho = tf.stack([tf.eye(2) for _ in range(4)])
init_sigma = tf.ones_like(samples['sigma'])
init_sigma_café = tf.ones_like(samples['sigma_alpha_beta'])
init_b = tf.ones_like(samples['beta'])
init_a = tf.ones_like(samples['alpha'])
init_mvn = tf.ones_like(samples['a_cafe_b_cafe'])
bijectors_list = [
    tfb.Exp(),
    tfb.Identity(),
    tfb.Exp(),
    tfb.CorrelationCholesky(),
    tfb.Identity(),
    tfb.Identity(),
]
m14_1.fit(observed_data = dict(y = d.wait.astype('float32').values),num_chains=4, 
          init= [init_sigma_café, init_b, init_a, init_rho, init_mvn, init_sigma], bijectors = bijectors_list)
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")

2024-03-12 14:56:37.517599: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 14:56:37.517668: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 14:56:37.517681: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 14:56:37.517835: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 14:56:37.517847: I tensorflow/core/common_runtime/gpu/gpu

BI took: 21.6142 seconds


In [141]:
import stan
import nest_asyncio
nest_asyncio.apply()
stan_code = """ 
data{
    vector[200] wait;
    array[200] int afternoon;
    array[200] int cafe;
}
parameters{
    vector[20] b_cafe;
    vector[20] a_cafe;
    real a;
    real b;
    vector<lower=0>[2] sigma_cafe;
    real<lower=0> sigma;
    corr_matrix[2] Rho;
}
model{
    vector[200] mu;
    Rho ~ lkj_corr( 2 );
    sigma ~ exponential( 1 );
    sigma_cafe ~ exponential( 1 );
    b ~ normal( -1 , 0.5 );    
    a ~ normal( 5 , 2 );
    {
        array[20] vector[2] YY;
        vector[2] MU;
        MU = [ a , b ]';
        for ( j in 1:20 ) YY[j] = [ a_cafe[j] , b_cafe[j] ]';
        YY ~ multi_normal( MU , quad_form_diag(Rho , sigma_cafe) );
    }
    for ( i in 1:200 ) {
        mu[i] = a_cafe[cafe[i]] + b_cafe[cafe[i]] * afternoon[i];
        
    }
    
    wait ~ normal( mu , sigma );
}
"""
data = {
    'wait' : d['wait'].values.astype(float),
    'afternoon' : d['afternoon'].values.astype(int),
    'cafe' : d['cafe'].values.astype(int)+1,
}
start = tm.time()
stan_model = stan.build(stan_code, data = data)
fit = stan_model.sample(num_chains=4, num_samples=2000, num_warmup = 500)
end = tm.time()    
df = fit.to_frame()
print(f"Pystan took: {end - start:.4f} seconds")

Building...



Building: 23.1s, done.Messages from stanc:
Warning in '/tmp/httpstan_uv1wdhmn/model_adioy7mi.stan', line 18, column 4: It
    is suggested to reparameterize your model to replace lkj_corr with
    lkj_corr_cholesky, the Cholesky factor variant. lkj_corr tends to run
    slower, consume more memory, and has higher risk of numerical errors.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
Sampling:   0%
Sampling:  25% (2500/10000)
Sampling:  50% (5000/10000)
Sampling:  75% (7500/10000)
Sampling: 100% (10000/10000)
Sampling: 100% (10000/10000), done.
Messages received during sampling:
  Gradient evaluation took 0.000205 seconds
  1000 transitions using 10 leapfrog steps per transition would take 2.05 seconds.
  Adjust your expectations accordingly!
  Informational Message: The current Metropolis proposal is about to be re

Pystan took: 25.7621 seconds


### 12.2. Output comparaison

In [143]:
pd.DataFrame(
    {
        "tfp": m14_1.summary(round_to='none')['mean'],
        "pystan": [df['sigma_cafe.1'].mean(),
                   df['sigma_cafe.2'].mean(),
                   df['b'].mean(),df['a'].mean(),
                   df['Rho.1.1'].mean(),df['Rho.2.1'].mean(),
                   df['Rho.1.2'].mean(),df['Rho.2.2'].mean(),
                   df['a_cafe.1'].mean(), df['b_cafe.1'].mean(),
                   df['a_cafe.2'].mean(), df['b_cafe.2'].mean(),
                   df['a_cafe.3'].mean(), df['b_cafe.3'].mean(),
                   df['a_cafe.4'].mean(), df['b_cafe.4'].mean(),
                   df['a_cafe.5'].mean(), df['b_cafe.5'].mean(),
                   df['a_cafe.6'].mean(), df['b_cafe.6'].mean(),
                   df['a_cafe.7'].mean(), df['b_cafe.7'].mean(),
                   df['a_cafe.8'].mean(), df['b_cafe.8'].mean(),
                   df['a_cafe.9'].mean(), df['b_cafe.9'].mean(),
                   df['a_cafe.10'].mean(), df['b_cafe.10'].mean(),
                   df['a_cafe.11'].mean(), df['b_cafe.11'].mean(),
                   df['a_cafe.12'].mean(), df['b_cafe.12'].mean(),
                   df['a_cafe.13'].mean(), df['b_cafe.13'].mean(),
                   df['a_cafe.14'].mean(), df['b_cafe.14'].mean(),
                   df['a_cafe.15'].mean(), df['b_cafe.15'].mean(),
                   df['a_cafe.16'].mean(), df['b_cafe.16'].mean(),
                   df['a_cafe.17'].mean(), df['b_cafe.17'].mean(),
                   df['a_cafe.18'].mean(), df['b_cafe.18'].mean(),
                   df['a_cafe.19'].mean(), df['b_cafe.19'].mean(),
                   df['a_cafe.20'].mean(), df['b_cafe.20'].mean(),
                   df['sigma'].mean()]
    })

,tfp,pystan
sigma_alpha_beta[0],1.018232,1.008108
sigma_alpha_beta[1],0.638057,0.636689
beta[0],-1.041135,-1.045726
alpha[0],3.677720,3.677060
"Rho[0, 0]",1.000000,1.000000
"Rho[0, 1]",0.000000,-0.637854
"Rho[1, 0]",-0.668078,-0.637854
"Rho[1, 1]",0.721207,1.000000
"a_cafe_b_cafe[0, 0]",3.037596,3.031840
"a_cafe_b_cafe[0, 1]",-0.973616,-0.976320
